# [문제]
- ChatPromptTemplate() : 역할 부여
- InMemoryChatMessageHistory()
- RunnableWithMessageHistory()
- chain 구성

## [문제] 기존 String PromptTemplate을 ChatPromptTemplate으로 변경
- Chain
- prompt template: 직접 작성
                   문서를 참고해서, 사용자 질문에 답변

In [ ]:
## 1. 모듈(파일, 라이브러리) 읽어오기
from langchain_openai import ChatOpenAI
from langchain_core.runnables.history import RunnableWithMessageHistory
from langchain_core.chat_history import InMemoryChatMessageHistory
from langchain_core.prompts import ChatPromptTemplate
from langchain.callbacks.streaming_stdout import StreamingStdOutCallbackHandler

from dotenv import load_dotenv


## 2. 환경변수에 설정된 값 읽어오기
load_dotenv()

## 3. 파일 읽기
file_name = '남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률(법률)(제20521호)(20250223).txt'

with open(file_name, 'r', encoding='utf-8') as file:
    law = file.read()
    print(law[:50])

## 4. prompt 생성
template = [
    ('system', '''당신은 육아휴직 법률 전문가입니다.
     아래 문서를 참고하여, 사용자 질문에 성실하게 답변합니다.
     답변은 문서 내용을 기반으로 하되, 해당 조항도 표시합니다.
     - 문서: {law}'''),
     ('placeholder', '{chat_history}'),
    ('user', '{query}')
]

prompt = ChatPromptTemplate.from_messages(template)

## 5. ChatOpenAI 인스턴스 생성
llm = ChatOpenAI(
    model='gpt-4.1-nano',
    streaming=True,
    callbacks=[StreamingStdOutCallbackHandler()],
)

## 6. chain 구성
chain = prompt | llm

## 7. 대화 이력(히스토리) 관리 인스턴스 생성
history = InMemoryChatMessageHistory()



남녀고용평등과 일ㆍ가정 양립 지원에 관한 법률 ( 약칭: 남녀고용평등법 )
[시행 20


In [6]:
store ={}

## 8. 함수 정의: 대화 이력(히스토리) 인스턴스 리턴
def get_history(session_id: str):
    if session_id not in store:
        store[session_id] = InMemoryChatMessageHistory()
    return store[session_id]

## 9. 사용자 질문 받기
query = input('육아휴직과 관련된 질문하세요 >> ')

## 10. chain에 대화 이력(히스토리) 기능을 래핑해서 추가
with_message_history = RunnableWithMessageHistory(
    chain,
    get_history,
    history_messages_key='chat_history',
    input_messages_key='query',
)

## 11. 실행
with_message_history.invoke(
    {'law': law, 'query': query},
    config={'configurable': {'session_id': 'abc'}}
)

남녀고용평등법 제19조(육아휴직)에 따라, 근로자가 임신 중이거나 만 8세 이하 또는 초등학교 2학년 이하 자녀를 양육하기 위해 휴직(육아휴직)을 신청하는 경우, 사업주는 이를 허용하여야 합니다. (제19조 1항)  
또한, 육아휴직의 기간은 1년 이내로 하되, 특정 조건을 만족하는 경우 최대 2년까지 가능하다고 명시되어 있습니다. (제19조 2항)  
육아휴직 기간 동안에는 근로자를 해고하거나 불리한 처우를 해서는 아니 되며, 복귀 시에는 휴직 전과 같은 직무 또는 수준의 임금을 지급하는 직무에 복귀시켜야 합니다. (제19조 3항, 4항)  
즉, 더 오래 휴식을 원하시면, 조건에 따라 최대 2년까지 육아휴직 연장이 가능하며, 이 기간 동안 해고 등 불이익 조치를 받지 않게 보호됩니다.  
추가로, 육아휴직 후 복귀 지원, 알선, 상담 등도 법적 의무 사항이니 참고하시기 바랍니다.

AIMessage(content='남녀고용평등법 제19조(육아휴직)에 따라, 근로자가 임신 중이거나 만 8세 이하 또는 초등학교 2학년 이하 자녀를 양육하기 위해 휴직(육아휴직)을 신청하는 경우, 사업주는 이를 허용하여야 합니다. (제19조 1항)  \n또한, 육아휴직의 기간은 1년 이내로 하되, 특정 조건을 만족하는 경우 최대 2년까지 가능하다고 명시되어 있습니다. (제19조 2항)  \n육아휴직 기간 동안에는 근로자를 해고하거나 불리한 처우를 해서는 아니 되며, 복귀 시에는 휴직 전과 같은 직무 또는 수준의 임금을 지급하는 직무에 복귀시켜야 합니다. (제19조 3항, 4항)  \n즉, 더 오래 휴식을 원하시면, 조건에 따라 최대 2년까지 육아휴직 연장이 가능하며, 이 기간 동안 해고 등 불이익 조치를 받지 않게 보호됩니다.  \n추가로, 육아휴직 후 복귀 지원, 알선, 상담 등도 법적 의무 사항이니 참고하시기 바랍니다.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_eede8f0d45'}, id='run-84084295-e405-4fea-92fd-cccec06dc49b-0')

In [8]:
get_history('abc')


InMemoryChatMessageHistory(messages=[HumanMessage(content='더쉬고 싶어 계속', additional_kwargs={}, response_metadata={}), AIMessage(content='남녀고용평등법 제19조(육아휴직)에 따라, 근로자가 임신 중이거나 만 8세 이하 또는 초등학교 2학년 이하 자녀를 양육하기 위해 휴직(육아휴직)을 신청하는 경우, 사업주는 이를 허용하여야 합니다. (제19조 1항)  \n또한, 육아휴직의 기간은 1년 이내로 하되, 특정 조건을 만족하는 경우 최대 2년까지 가능하다고 명시되어 있습니다. (제19조 2항)  \n육아휴직 기간 동안에는 근로자를 해고하거나 불리한 처우를 해서는 아니 되며, 복귀 시에는 휴직 전과 같은 직무 또는 수준의 임금을 지급하는 직무에 복귀시켜야 합니다. (제19조 3항, 4항)  \n즉, 더 오래 휴식을 원하시면, 조건에 따라 최대 2년까지 육아휴직 연장이 가능하며, 이 기간 동안 해고 등 불이익 조치를 받지 않게 보호됩니다.  \n추가로, 육아휴직 후 복귀 지원, 알선, 상담 등도 법적 의무 사항이니 참고하시기 바랍니다.', additional_kwargs={}, response_metadata={'finish_reason': 'stop', 'model_name': 'gpt-4.1-nano-2025-04-14', 'system_fingerprint': 'fp_eede8f0d45'}, id='run-84084295-e405-4fea-92fd-cccec06dc49b-0')])